In [3]:
from ase.io import read
from utils.visualization.visualizator import sith_viewer

In [2]:
def output_terminal(cmd):
    p = subprocess.Popen(cmd,
                         shell=True,
                         stdout=subprocess.PIPE)

    out, err = p.communicate()
    return out.decode('ascii')
    
def optimized_e(file):
    out = output_terminal('grep "E(RBMK) =" '+file)
    energy = float(out.split()[-5])
    return energy*27.21 # energy in eV

def distance(file, index1, index2):
    out = output_terminal(f'python /hits/basement/mbm/sucerquia/utils/distance.py {file} {index1} {index2}')
    return float(out)

def gpaw_e(file):
    out = output_terminal('grep Difference '+file)
    energy = float(out.split()[-1])
    return energy

def get_internal_coordinates(logfile, dims):
    out = output_terminal(f'grep -iA {dims+4} optimized {logfile} | tail -n {dims}')
    return out

def frozen_coords(file):
    out = output_terminal(f'grep -i "frozen" {file}')
    return list(set(out.split()[2::8]))
    
def compare_labels(optimized, deformed, n, m):
    '''optimized is the .log file to the optimized structure
       deformed is the .log file of the deformed structure
       n in the number of DOF of the optimized structure
       m is the number of DOF of the deformed structure'''               
    optimized_labels = get_internal_coordinates(optimized, n).split()[2::8]
    deformed_labels = get_internal_coordinates(deformed, m).split()[2::8]
    frozen = frozen_coords(deformed)
    indexes = [deformed_labels.index(label) for label in frozen]
    [deformed_labels.remove(DOF) for DOF in frozen]
    
    assert (np.array(optimized_labels) == np.array(deformed_labels)).all(), \
           f"The order in the paramenters is not the same, {np.array(optimized_labels)[np.invert(np.array(optimized_labels) == np.array(deformed_labels))]}"
    return indexes

def remove_additional_DOF(geo_object, indexes):
    dims = geo_object.dims
    for i in indexes:
        if i < dims[1]:
            geo_object.lengths.pop(i)
        elif i < dims[1] + dims[2]:
            i -= dims[1]
            geo_object.lengths.pop(i)
        else:
            i -= dims[1] + dims[2]
            geo_object.lengths.pop(i)


# Visualization

In [4]:
atoms = read('Ala-streched0.pdb')

atoms.center(vacuum=1)
viewer = sith_viewer(atoms)
v = viewer.viewer
v

In [6]:
viewer.add_dihedral(8, 6, 4, 5, color=[0,0,0], n=5)
viewer.add_bond(8, 6, [1, 0, 0], 0.02)
viewer.add_bond(4, 6, [1, 0, 0], 0.02)
viewer.add_bond(4, 5, [1, 0, 0], 0.02);

In [7]:
v.view.picked

{}

In [ ]:
v.view.center()
viewer.remove_bonds()
viewer.add_bond(2, 3, [1, 0, 0], 0.02)


In [ ]:
viewer.add_angle(3, 5, 4, color=[1,0,0], n=5)
viewer.add_angle(0, 1, 2, color=[0,1,0], n=5)
viewer.add_angle(1, 2, 3, color=[0,0,1], n=5)

In [ ]:
viewer.remove_all_angles()

In [ ]:
viewer.remove_angle(3, 5, 4)

In [ ]:
viewer.remove_bonds([5])